<div style="color:Navy"> 

<div style="text-align:center"> 

***
# <u>TP5:</u>
# Segmentation d'images
    
<p style="text-align: center; color:gray"><i>@Author:</i> Marc-Aurèle Rivière</p>

***
        
</div>
    
<u>**Plan:**</u>

1. [**Segmentation par seuillage simple**](#1): 
    * En luminance: thresholding, méthode d'Otsu, méthodes adaptatives
    * En chrominance

    
2. [**Segmentation non-supervisée**](#2): Mean-Shift, Watershed, Superpixels
    
</div>

In [2]:
'''''''''''''''''''''''''''''''''
#################################
#  Code global pour tout le TP  #
#################################
'''''''''''''''''''''''''''''''''

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("Running on Colaboratory")
    from google.colab import drive, files

    drive.mount('/content/gdrive', force_remount=True)
    root_path = 'gdrive/My Drive/3. Doctorat/Enseignements/[Intro] Image Processing/TP5/'  # A modifier à votre chemin d'accès
    img_path = root_path + "img/"
else:
    print("Not running on Colaboratory")
    root_path = "/"
    img_path = "img/"

Not running on Colaboratory


# <span style="color: Green;text-decoration: underline" id="1">I. Segmentation par seuillage simple</span>
***

**Seuiller** une image (*thresholding*) consiste à discrétiser l'échelle des valeurs de luminance de ses pixels selon un (ou plusieurs) seuils donnés.
- Tout pixel avec une valeure supérieure au seuil deviendra 255 (blanc)
- Tout pixel avec une valeure inférieure au seuil deviendra 0 (noir)

Il existe **plusieurs types d'algorithmes de segmentation**:
* __(Supervisé, bottom-up):__ nécessitent de fournir les seuils manuellement: _thresholding_
* __(Non-supervisé, bottom-up):__ déterminent les seuils à partir du contenu de l'image (luminance, chrominance)
  * Thresholding non-supervisé: _adaptative thresholding, Otsu's method_
  * Méthodes de clustering: _mean-shift (QuickShift), watershed, super-pixels (SLIC)_
*  __(Supervisé, top-down):__ déterminent les frontières de segmentation à partir de classes ayant un sens sémantique (généralement définies par l'Homme), apprises lors d'une phase d'entrainement
  * Machine Learning: _K-NN, SVM,..._
  * Deep Learning: _U-Net, Mask-R-CNN, Deeplab,..._

<img src="https://miro.medium.com/max/2001/1*zKnOz-YWIKtIohhYcydNEQ.png">


Pour cette première partie, nous allons nous focaliser sur les solutions de **segmentation par seuillage (supervisées ou non)**, en luminance et en chrominance.

In [13]:
### Imports et fonctions utiles à cette partie

import os, cv2
import numpy as np
from PIL import Image, ImageOps
from skimage.morphology import erosion, closing, square, remove_small_holes
from skimage.segmentation import clear_border
from skimage.filters import threshold_multiotsu
from skimage.measure import label, regionprops
from skimage.color import label2rgb

from matplotlib.pylab import *
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb, rgb_to_hsv
%matplotlib inline
# Améliorer la netteté des figures
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = 12.8, 9.6

# "Do not disturb" mode
import warnings                                        
warnings.filterwarnings('ignore')

# Interactivité
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive_output
from IPython.display import display, Markdown
import emoji

'''
Méthodes d'affichage
'''
def affichage_auto_simple(results, titles, cm="viridis"):
    size = len(results)
    plt.figure(figsize=(6*size, 6))
    for j, _res in enumerate(results):
        plt.subplot(f"1{size}{j+1}")
        plt.imshow(_res, cmap=cm, origin="upper")
        plt.title(titles[j] if titles != None and j < len(titles) else f"Image {j+1}")
    plt.show()


def affichage_auto(original, processed, titles, breakpoint=3, cm="gray"):
    row = breakpoint
    col = int(np.ceil(len(processed)/float(row)))
    
    # Affichage de l'image d'origine et de son histogramme
    fig1 = plt.figure(figsize=(12, 6))
    plt.subplot(121), plt.imshow(original, cmap=cm, origin="upper"), plt.title("Original")
    plt.subplot(122), plt.hist(np.array(original).ravel(), 255, [0,255]), plt.title("Histogramme")
    
    # Affichage des différentes versions segmentées 
    fig2 = plt.figure(figsize=(5*row, 5*col))
    for i, img in enumerate(processed):
        plt.subplot(f"{col}{row}{i+1}")
        plt.imshow(img, cmap=cm, origin="upper")
        plt.title(titles[i] if titles != None and i < len(titles) else f"Image {i}")
    plt.show()
    
    
def affichage_otsu(original, result, thresh):
    plt.figure(figsize=(18, 6))
    plt.subplot(131), plt.imshow(original, cmap=cm, origin="upper"), plt.title("Original")
    ax2 = plt.subplot(132)
    ax2.hist(np.array(original).ravel(), 255, [0,255])
    ax2.set_title("Histogramme")
    ax2.axvline(thresh, color='r')
    plt.subplot(133), plt.imshow(result, cmap=cm, origin="upper"), plt.title("Original")
    plt.show()


def affichage_masks_and_bounds(original, bounds, masks):
    
    size_b = len(bounds) * 2
    size_m = len(masks) + 1
    
    fig1 = plt.figure(figsize=(5*size_m, 5))
    plt.subplot(f"1{size_m}1"), plt.imshow(original, origin="upper"), plt.title("Original")
    for j, _mask in enumerate(masks):
        plt.subplot(f"1{size_m}{j+2}"), plt.imshow(_mask, cmap="gray"), plt.title(f"Mask {j+1}")
    
    fig2 = plt.figure(figsize=(3*size_b, 3))
    c = 1
    for i, _bound in enumerate(bounds):
        lower = np.full((10, 10, 3), _bound[0]).astype("uint8") / 255.0
        upper = np.full((10, 10, 3), _bound[1]).astype("uint8") / 255.0
        plt.subplot(f"1{size_b}{c}"), plt.imshow(hsv_to_rgb(lower)), plt.title(f"Lower {i+1}")
        plt.subplot(f"1{size_b}{c+1}"), plt.imshow(hsv_to_rgb(upper)), plt.title(f"Upper {i+1}")
        c+=2
    
    plt.show()

## <span style="color: DodgerBlue;text-decoration: underline">I.1 Segmentation en luminance</span>

### I.1.a Thresholding:

#### Binarisation avec OpenCV:

```Python
_, imageBinaire = cv2.threshold(img_gris, seuil, nouvelle_valeur, cv2.THRESH_BINARY)
```

**Avec:**
- ```image```: l'image source (en niveaux de gris)
- ```seuil```: seuil de binarisation, généralement (255-1)/2 = 127
- ```nouvelle_valeur```: nouvelle valeur que vont prendre les pixels (généralement 255)

Tous les pixels dont la valeur est > à 127 prendront la valeur 255, et la valeur de 0 pour les autres.

*OpenCV provides different types of thresholding which is given by the fourth parameter of the function. Basic thresholding as described above is done by using the type cv.THRESH_BINARY. All simple thresholding types are:*

* `cv.THRESH_BINARY`
* `cv.THRESH_BINARY_INV`
* `cv.THRESH_TRUNC`
* `cv.THRESH_TOZERO`
* `cv.THRESH_TOZERO_INV`

<img src="https://miro.medium.com/max/1730/1*gmOL367EAnlsSdtNFqjB-A.png">
<img src="https://miro.medium.com/max/1095/1*swjBYQOnuNfv1rHM3p39PQ.png">

In [7]:
## Comparaison des différents types de segmentation simple d'OpenCV

images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='adapt2.jfif', description='Image:')
seuil_slider = widgets.IntSlider(min=50, max=250, step=5, value=127, continuous_update=False)

@interact
def binarisation(image=images_dropdown, seuil=seuil_slider):
    
    img = np.array(Image.open(img_path + image).convert("L")).astype("uint8")
    
    _,thresh1 = cv2.threshold(img,seuil,255,cv2.THRESH_BINARY)
    _,thresh2 = cv2.threshold(img,seuil,255,cv2.THRESH_BINARY_INV)
    _,thresh3 = cv2.threshold(img,seuil,255,cv2.THRESH_TRUNC)
    _,thresh4 = cv2.threshold(img,seuil,255,cv2.THRESH_TOZERO)
    _,thresh5 = cv2.threshold(img,seuil,255,cv2.THRESH_TOZERO_INV)

    titles = ['BINARY','BINARY_INV','TRUNC','TOZERO','TOZERO_INV']
    images = [thresh1, thresh2, thresh3, thresh4, thresh5]

    affichage_auto(img, images, titles)

interactive(children=(Dropdown(description='Image:', index=1, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

### I.1.b Otsu's thresholding :

La méthode d'Otsu est une méthode de **seuillage simple non-supervisé** qui permet de déterminer la valeur du seuil global de binarisation en analysant l'histogramme de l'image N&B: l'algorithme choisira automatiquement le seuil de sorte à rendre la répartition des pixels de chaque côté de la frontière la plus équitable possible.

<img src="https://scikit-image.org/docs/dev/_images/sphx_glr_plot_thresholding_0011.png">

#### Avec OpenCV:

Il suffit de rajouter le "flag" `cv2.THRESH_OTSU` à l'une des méthodes de binarisation d'OpenCV.

In [11]:
seuil_slider = widgets.IntSlider(min=101, max=201, step=2, value=127, continuous_update=False)
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='cells.png', description='Image:')

@interact
def otsu(image=images_dropdown, seuil=seuil_slider):
    img = np.array(Image.open(img_path + image).convert("L")).astype("uint8")

    # Global thresholding
    ret1,th1 = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)

    # Otsu's thresholding
    ret2,th2 = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Otsu's thresholding after Gaussian filtering
    blur = cv2.GaussianBlur(img, (5,5), 0)
    ret3,th3 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    images = [th1, th2, th3]
    titles = [f'Global Thresholding: {ret1}', f"Otsu's Thresholding: {ret2}", f"Otsu's Thresholding after Gaussian Filt.: {ret3}"]
    affichage_auto(img, images, titles)

interactive(children=(Dropdown(description='Image:', index=3, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

#### Multiple Otsu avec `Skimage`:

Il est possible de déterminer plusieurs seuils via la méthode d'Otsu, afin d'avoir une segmentation qui conserve plus d'informations sur la répartition des niveaux de luminance de l'image d'origine.

In [10]:
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='cells.png', description='Image:')

@interact
def otsu(image=images_dropdown):
    img = np.array(Image.open(img_path + image).convert("L")).astype("uint8")

    thresholds = threshold_multiotsu(img)

    # Using the threshold values, we generate the three regions.
    regions = np.digitize(img, bins=thresholds)
    
    affichage_auto_simple([img, regions], ["Original", "Multiple Otsu"], cm="gray")

interactive(children=(Dropdown(description='Image:', index=3, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

### I.1.c Adaptative Thresholding:

Dans les sections précédentes, l'on c'est limité à une valeur globale pour binariser toute l'image (*global thresholding*). Dans certain cas (e.g. quand l'illumination de l'image varie énormement dans différentes sections de l'image), cela produira de mauvais résultats. La binarisation adaptative (*adaptative thresholding*) permet de remédier à ce problème en **déterminant le seuil de manière locale** (en se basant sur un certain nombre de voisins d'un pixel donné). Il en résultera que différentes régions de l'image seront binarisés à différents seuils, en **prenant en compte le contexte local de luminance**.

```python
cv2.AdaptiveThreshold(src, [dst], maxValue, adaptive_method, thresholdType, blockSize, c)
```

<u>Avec</u> :
* `maxValue`: nouvelle valeur pour les pixels qui passent le seuil déterminé par la méthode adaptative (généralement 255).
* `adaptiveMethod`: méthode à utiliser pour déterminer le seuil (`ADAPTIVE_THRESH_MEAN_C` ou `ADAPTIVE_THRESH_GAUSSIAN_C`).
* `thresholdType`: type de binarisation à effectuer une fois le seuil déterminé (`THRESH_BINARY` ou `THRESH_BINARY_INV`).
* `blockSize`: taille du voisinnage évalué pour déterminer le seuil (matrice carrée impaire: 3x3, 5x5, ...)
* `C`: (optionel) constante soustraite de chaque valeur de seuil déterminé pour les différentes régions.
  * _`cv2.ADAPTIVE_THRESH_MEAN_C`: seuil = moyenne des valeurs de luminance des pixels voisins, moins `C`._
  * _`cv2.ADAPTIVE_THRESH_GAUSSIAN_C`: seuil = moyenne pondérée (selon un kernel Gaussien) des pixels voisins, moins `C`._

In [8]:
c_slider = widgets.IntSlider(min=0, max=125, step=5, value=5, continuous_update=False)
kernel_size_slider = widgets.IntSlider(min=3, max=29, step=2, value=11, continuous_update=False)
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='adapt2.jfif', description='Image:')

@interact
def adaptative_thresholding(image=images_dropdown, kernel_size=kernel_size_slider, c=c_slider):
    img = np.array(Image.open(img_path + image).convert("L")).astype("uint8")

    ret,th1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    th2 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, kernel_size, c)
    th3 = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, kernel_size, c)

    images = [th1, th2, th3]
    titles = [f'Global Thresholding: {ret}', 'Adaptive Mean Thresholding', 'Adaptive Gaussian Thresholding']
    affichage_auto(img, images, titles, breakpoint=4)

interactive(children=(Dropdown(description='Image:', index=1, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

## <span style="color: DodgerBlue;text-decoration: underline">I.2 Segmentation en chrominance</span>

La **segmentation en chrominance** étends les principes précédents aux images couleurs en les segmentant selon 3 seuils (pour chaque canal de l'image). 

Cette segmentation est généralement faite dans un espace colorimétrique permet de manipuler chrominance et luminance séparément, afin de pouvoir spécifier des seuils de couleurs qui fonctionneront pour un grand nombres de valeurs de luminance.

La plupart des méthodes de segmentation colorimétriques demandent deux seuils pour chaque channel: une borne inférieur et supérieur.

<u>Remarque</u>: outil utile pour prélever les couleurs d'une image: https://pinetools.com/image-color-picker

#### Avec un intervalle:

In [24]:
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='coins2.jpg', description='Image:')
h_slider = widgets.IntRangeSlider(value=[0, 179], min=0, max=179, step=1, description='Hue:', continuous_update=False)
s_slider = widgets.IntRangeSlider(value=[18, 255], min=0, max=255, step=1, description='Saturation:', continuous_update=False)
v_slider = widgets.IntRangeSlider(value=[18, 255], min=0, max=255, step=1, description='Value:', continuous_update=False)


@interact
def chrom_segment(image=images_dropdown, seuil_h=h_slider, seuil_s=s_slider, seuil_v=v_slider):
    img = np.array(Image.open(img_path + image)).astype("uint8")
    
    try:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    except:
        print("Selectionner une image couleur !")
        return
    
    lower_bound = np.array([seuil_h[0], seuil_s[0], seuil_v[0]])
    upper_bound = np.array([seuil_h[1], seuil_s[1], seuil_v[1]])
    mask = cv2.inRange(img_hsv, lower_bound, upper_bound)
    
    # Nettoyage du mask par opérations morphologiques (skimage)
    mask = remove_small_holes(mask, area_threshold=500)
    mask = cv2.morphologyEx(np.array(mask).astype("uint8"), cv2.MORPH_CLOSE, np.ones((3,3), dtype="int"))
    mask = clear_border(mask)
    mask = cv2.erode(np.array(mask).astype("uint8"), np.ones((3,3), dtype="int"))
    mask = np.array(mask).astype("uint8")
    
    affichage_masks_and_bounds(img, [(lower_bound, upper_bound)], [mask])
    
    segment_button = widgets.Button(description='Segmenter', icon="check", style=widgets.ButtonStyle(button_color='lightblue'))
    display(segment_button)
    display(Markdown(emoji.emojize("👆 Cliquez sur le bouton 'Segmenter' une fois vos seuils sélectionnés pour appliquer le masque")))
    
    contour_button = widgets.Button(description='Bounding Box', icon="check", style=widgets.ButtonStyle(button_color='lightgreen'))
    display(contour_button)
    display(Markdown(emoji.emojize("👆 Cliquez sur le bouton 'Bounding Box' une fois vos seuils sélectionnés pour créer les Bounding Boxes")))
    
    @segment_button.on_click
    def do_segment(b):
        masked = cv2.bitwise_and(img, img, mask=mask)
        affichage_auto_simple([masked], ["Masque de chrominance"])
    
    @contour_button.on_click
    def do_contours(b):
        labels, n_labels = label(mask, return_num=True)
        labeled_img = label2rgb(labels, image=img)
        
        props = regionprops(labels)
        contoured = np.array(img.copy())
        
        for prop in props:
            if prop.bbox_area > 0:
                cv2.rectangle(contoured, (prop.bbox[1], prop.bbox[0]), (prop.bbox[3], prop.bbox[2]), (145, 0, 0), 2)

        affichage_auto_simple([labeled_img, contoured], [f"Labels: {n_labels}", f"Bounding boxes: {n_labels}"])

interactive(children=(Dropdown(description='Image:', index=5, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

#### Avec deux intervalles:

In [7]:
'''
Création de l'UI pour l'interaction avec l'utilisateur
'''
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='nemo.jpg', description='Image:')

h_slider1 = widgets.IntRangeSlider(value=[1, 20], min=0, max=179, step=1, description='Hue:', continuous_update=False)
s_slider1 = widgets.IntRangeSlider(value=[190, 255], min=0, max=255, step=1, description='Saturation:', continuous_update=False)
v_slider1 = widgets.IntRangeSlider(value=[110, 255], min=0, max=255, step=1, description='Value:', continuous_update=False)
box1 = widgets.VBox(children=[h_slider1, s_slider1, v_slider1])

h_slider2 = widgets.IntRangeSlider(value=[0, 179], min=0, max=179, step=1, description='Hue:', continuous_update=False)
s_slider2 = widgets.IntRangeSlider(value=[0, 90], min=0, max=255, step=1, description='Saturation:', continuous_update=False)
v_slider2 = widgets.IntRangeSlider(value=[170, 255], min=0, max=255, step=1, description='Value:', continuous_update=False)
box2 = widgets.VBox(children=[h_slider2, s_slider2, v_slider2])

app = widgets.AppLayout(header=images_dropdown,
          left_sidebar=box1,
          center=None,
          right_sidebar=box2,
          footer=None,
          justify_items='center', align_items='center')


def clean_mask(mask, holes_thresh=200):
    mask = remove_small_holes(mask, area_threshold=holes_thresh)
    #mask = closing(mask, square(3))
    mask = cv2.morphologyEx(np.array(mask).astype("uint8"), cv2.MORPH_CLOSE, np.ones((3,3), dtype="int"))
    mask = clear_border(mask)
    #mask = erosion(mask, square(3))
    mask = cv2.erode(np.array(mask).astype("uint8"), np.ones((3,3), dtype="int"))
    return np.array(mask).astype("uint8")

'''
Fonction principale
'''
def chrom_segment2(image, h1,s1,v1, h2,v2,s2):
    img = np.array(Image.open(img_path + image)).astype("uint8")
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Orange
    lower1 = np.array([h1[0], s1[0], v1[0]])
    upper1 = np.array([h1[1], s1[1], v1[1]])
    # Blanc
    lower2 = np.array([h2[0], s2[0], v2[0]])
    upper2 = np.array([h2[1], s2[1], v2[1]])

    mask1 = cv2.inRange(img_hsv, lower1, upper1)
    mask2 = cv2.inRange(img_hsv, lower2, upper2)
    mask1 = clean_mask(mask1, holes_thresh=500)
    mask2 = clean_mask(mask2, holes_thresh=500)
    
    affichage_masks_and_bounds(img, [(lower1,upper1), (lower2,upper2)], [mask1, mask2])
    
    mask = mask1 + mask2
    result = cv2.bitwise_and(img, img, mask=mask)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    final = cv2.drawContours(img, contours, -1, (0,255,0), 3)
    
    images = [result, final]
    titles = ["Segmenté","Avec frontières"]
    affichage_auto_simple(images, titles)
    
'''
On associe la fonction principale à l'UI pour réagir aux modifications de cette dernière
'''
out = interactive_output(chrom_segment2, {"image":images_dropdown, "h1":h_slider1, "s1":s_slider1, "v1":v_slider1, 
                                          "h2":h_slider2, "s2":s_slider2, "v2":v_slider2})
display(app, out)

AppLayout(children=(Dropdown(description='Image:', index=9, layout=Layout(grid_area='header'), options=('adapt…

Output()

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">
    
<span style="color:green">**Le but de cet exercice est de générer un masque de segmentation pour une image, et de compter le nombre d'éléments qu'elle contient grâce à ce masque.**</span>
    
1. **Effectuer un seuillage binaire sur l'image `coins3.jpg`:**
    * Vous pouvez épurer l'image d'origine et/ou le masque avec les opérations morphologiques appropriées.
    * Vous pouvez éventuellement réduire les différences de luminance au sein des pièces par des transformations de niveau de gris (min-max, egalisation / AHE, log, ...)
 
    
2. **Répétez l'opération en utilisant un seuillage par intervalle de niveaux de gris.**
    
    
3. **En vous basant sur le second exemple de la section I.2, élaborez deux masques de segmentation de couleur pour l'image `coins2.jpg`, permettant chacun d'extraire une couleur de pièce..**
    * Ajoutez une option interactive permettant d'appliquer au choix l'un ou les deux masques à l'image.
    

4. **A partir de la combinaison des deux masques précédents, traçez les contours des pièces.**
        

5. **Calculez la superficie des pièces segmentées.**

    > <u>Astuce</u>:
    ```python
    for cnt in contours:
        area = cv2.contourArea(cnt)
    ```
   
6. **Faites encadrer les pièces par une enveloppe englobante ellipsoidale et les afficher sur l'image d'origine.**
    
    > <u>Astuce</u>:
    ```python
    ellipse = cv2.fitEllipse(cnt)
    cv2.ellipse(img, ellipse, (0,255,0), 2)
    ```

    
7. **Dans une nouvelle liste, réorganiser les pièces par taille croissante et afficher un chiffre sur la pièce représentant sa place dans cette liste.**
    
    > <u>Astuce</u>: vous pouvez écrire au centre de l'ellipse correspondante (`ellipse.center`).


8. **Modifiez votre code de sorte à ce qu'il fonctionne également pour l'image des cellules, et appliquez-le à celle-ci afin de les segmenter, compter et mesurer.**
    
    > <u>Remarque</u>: la superficie extraite ainsi pourrait servir de feature basique pour une méthode de Machine Learning permettant de classifier / clusteriser les cellules.
    
</div>

In [29]:
# > Emplacement exercice <



# <span style="color: Green;text-decoration: underline" id="2">II. Segmentation non-supervisée</span>
***

Dans cette section, nous allons nous intéresser aux **méthodes automatisées ou semi-automatisées** de segmentation d'images, qui reposent généralement sur des algorithmes de **clustering**.

Ces méthodes sont **non (ou faiblement) supervisées** et se basent également sur des indices bas-niveau (*bottom-up*), comme la luminance et la chrominance, afin de déterminer les frontières de segmentation les plus pertinentes entre les pixels. Ce sont des méthodes locales qui vont générer des frontières multiples en comparant la valeur des pixels pour les regrouper en régions "similaires". L'objectif de ces algorithmes est d'optimiser progressivement les régions générées afin qu'elles correspondent à des régions ou objets naturelles, i.e. qui ont une **signification sémantique** pour l'Homme.

L'utilisateur n'a plus de valeurs de seuils à fournir, mais généralement un certain nombre de méta-paramètres qui vont guider l'algorithme dans sa recherche des seuils optimaux: nombre de culters, points de départ de la recherche (*markers*), ...

<img src="https://filebox.ece.vt.edu/~jbhuang/teaching/ece5554-4554/fa16/images/slic.jpg">


Quelques exemples d'algorithmes:
* Mean-Shift / Quickshift
* Superpixels (*SLIC*)
* Watershed

In [3]:
'''
Imports et fonctions utiles à cette partie
'''

import os, cv2
import numpy as np
from PIL import Image, ImageOps
from skimage.segmentation import slic, quickshift, watershed, mark_boundaries
from skimage.color import label2rgb

from matplotlib.pylab import *
import matplotlib.pyplot as plt
%matplotlib inline
# Améliorer la netteté des figures
%config InlineBackend.figure_format = 'retina'
plt.rcParams["figure.figsize"] = 12.8, 9.6

# "Do not disturb" mode
import warnings                                        
warnings.filterwarnings('ignore')

# Interactivité
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, interactive_output

'''
Méthodes d'affichage
'''
def affichage_auto_simple(results, titles, cm="viridis"):
    size = len(results)
    plt.figure(figsize=(6*size, 6))
    for j, _res in enumerate(results):
        plt.subplot(f"1{size}{j+1}")
        plt.imshow(_res, cmap=cm, origin="upper")
        plt.title(titles[j] if titles != None and j < len(titles) else f"Image {j+1}")
    plt.show()
    

def affichage_auto(original, processed, titles, breakpoint=3, cm="gray"):
    row = breakpoint
    col = int(np.ceil(len(processed)/float(row)))
    
    # Affichage de l'image d'origine et de son histogramme
    fig1 = plt.figure(figsize=(6, 6))
    plt.subplot(111), plt.imshow(original, cmap=cm, origin="upper"), plt.title("Original")
    
    # Affichage des différentes versions segmentées 
    fig2 = plt.figure(figsize=(5*row, 5*col))
    for i, img in enumerate(processed):
        plt.subplot(f"{col}{row}{i+1}")
        plt.imshow(img, cmap=cm, origin="upper")
        plt.title(titles[i] if titles != None and i < len(titles) else f"Image {i}")
    plt.show()

## <span style="color: DodgerBlue;text-decoration: underline">II.1 Mean-Shift filtering</span>

**_Mean shift filtering_** est un algorithme de culstering fréquemment utilisé en traitement d'image. C'est un algorithme de recherche du "mode" local (d'un groupe de pixel) afin d'homogénéiser ce groupe selon la valeur du mode. Il va "applatir" les gradients (hautes-fréquences) de texture (spatial) et de couleur.

*For each pixel of an image (having a spatial location and a particular color), the set of neighboring pixels (within a spatial radius and a defined color distance) is determined. For this set of neighbor pixels, the new spatial center (spatial mean) and the new color mean value are calculated. These calculated mean values will serve as the new center for the next iteration. The described procedure will be iterated until the spatial and the color (or grayscale) mean stops changing. At the end of the iteration, the final mean color will be assigned to the starting position of that iteration.*

<img src="https://i.stack.imgur.com/E9ItQ.png">

*The Mean Shift algorithm takes usually 3 inputs:*

* _**A distance / similarity measure** for comparing pixels: L1 (Manhattan) or L2 (Euclidian) distance usually._
* _**A radius / kernel size**: all pixels within this radius (measured according the above distance) will be accounted for the calculation._
* _**A max value difference**: from all pixels inside the radius, we will take only those whose values are within this difference for calculating the mean._

### Avec `OpenCV`:

```python
shifted = cv2.PyrMeanShiftFiltering(src, sp, sr)
```

Avec:
* `sp`: la taille du kernel spatial.
* `sr`: la taille du kernel de couleur.

[Documentation Mean-Shift](https://docs.opencv.org/2.4/modules/imgproc/doc/filtering.html#pyrmeanshiftfiltering)

In [4]:
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='coins2.jpg', description='Image:')
color_kernel_size_slider = widgets.IntSlider(min=5, max=71, step=2, value=51, continuous_update=False)
kernel_size_slider = widgets.IntSlider(min=5, max=35, step=2, value=21, continuous_update=False)

@interact
def mean_shift(image=images_dropdown, kernel_size=kernel_size_slider, color_kernel_size=color_kernel_size_slider):
    img = np.array(Image.open(img_path + image)).astype("uint8")

    cv2_mean_shift = cv2.pyrMeanShiftFiltering(img, sp=kernel_size, sr=color_kernel_size)

    affichage_auto_simple([img, cv2_mean_shift], ["Original", "Mean Shift"])

interactive(children=(Dropdown(description='Image:', index=5, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

### Avec `Skimage`: 

_Quickshift has 4 main parameters:_
* _`kernel_size` controls the size of the kernel used in smoothing the sample density. Higher means fewer clusters._
* _`max_dist` controls the cut-off point for data distances. Higher means fewer clusters._
* _`ratio` controls the trade-off between color-space proximity and image-space proximity. Higher values give more weight to color-space._
* _`sigma` controls the width of the Gaussian smoothing as preprocessing (zero means no smoothing)._

[Documentation Quickshift](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.quickshift)

In [5]:
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='coins2.jpg', description='Image:')
kernel_size_slider = widgets.IntSlider(min=3, max=25, step=2, value=11, continuous_update=False)
distance_slider = widgets.IntSlider(min=5, max=71, step=2, value=41, continuous_update=False)
ratio_slider = widgets.FloatSlider(min=0.1, max=1.0, step=0.1, value=0.5, continuous_update=False)
smoothing_slider = widgets.FloatSlider(min=0.0, max=20.0, step=1.0, value=5.0, continuous_update=False)


@interact
def quickshift(image=images_dropdown, kernel_s=kernel_size_slider, max_d=distance_slider, ratio_val=ratio_slider, smoothing_val=smoothing_slider):
    img = np.array(Image.open(img_path + image)).astype("uint8")
    
    segments_quick = quickshift(img, kernel_size=kernel_s, max_dist=max_d, sigma=smoothing_val)
    img_quick = mark_boundaries(img, segments_quick)
    homogenized_quick = label2rgb(segments_quick, img, kind='avg')
    
    affichage_auto_simple([img, img_quick, homogenized_quick], ["Original", f"Quickshift clusters: {np.unique(segments_quick).size}", "Homogenized"])

interactive(children=(Dropdown(description='Image:', index=5, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

## <span style="color: DodgerBlue;text-decoration: underline">II.2 Watershed</span>

*The **watershed algorithm** is a classic algorithm used for segmentation and is especially useful when extracting touching or overlapping objects in images.*

<img src="https://opencv-python-tutroals.readthedocs.io/en/latest/_images/water_result.jpg">

*Instead of taking a color image as input, watershed requires a grayscale gradient image, where bright pixels denote a boundary between regions. The algorithm views the image as a landscape, with bright pixels forming high peaks. Indeed, any grayscale image can be viewed as a topographic surface where high intensity denotes peaks and hills while low intensity denotes valleys.*

*Then, you start filling every isolated valleys (local minima) with different colored water (labels). As the water rises, depending on the peaks (gradients) nearby, water from different valleys (with different colors) will start to merge. To avoid that, you build barriers in the locations where water merges. You continue the work of filling water and building barriers until all the peaks are under water. Then the barriers you created gives you the segmentation result.*

<img src="https://imagej.net/_images/thumb/c/c5/Watershed-flooding-graph.png/375px-Watershed-flooding-graph.png">

*When utilizing the watershed algorithm we must start with user-defined markers. These markers can be either manually defined via point-and-click, or we can automatically or heuristically define them using methods such as thresholding and/or morphological operations (such as the distance transform).*

*Based on these markers, the watershed algorithm treats pixels in our input image as local elevation (called a topography) — the method “floods” valleys, starting from the markers and moving outwards, until the valleys of different markers meet each other. In order to obtain an accurate watershed segmentation, those markers must be correctly placed : the initialisation phase has a huge impact on the algorithm results.*

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn%3AANd9GcQxRc8v4c1VqKqMpGaq5qFW0FPiIFm4i9uVdnY-rThDki_9gzFV">

### Avec `Skimage`:

_Watershed has 2 main parameters:_
* _`markers` controls the desired number of markers, or an array marking the basins with the values to be assigned as markers -where zero means 'not a marker'). If `None` (no markers given), the local minima of the image are automatically used as markers._
* _`compactness` controls the shape and overall distance between a pixel and its cluster center. Higher values result in more regularly-shaped watershed basins (usually squared)._

[Documentation Watershed](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.watershed)

In [8]:
from skimage.filters import sobel
from skimage.color import rgb2gray

images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='coins2.jpg', description='Image:')
markers_slider = widgets.IntSlider(min=10, max=300, step=10, value=100, continuous_update=False)
compactness_slider = widgets.FloatSlider(min=1, max=10, step=1, value=1, continuous_update=False)

@interact
def watershed(image=images_dropdown, n_markers=markers_slider, compact=compactness_slider):
    img = np.array(Image.open(img_path + image)).astype("uint8")
    img_sobel = sobel(rgb2gray(img))
    
    segments_watershed = watershed(img_sobel, markers=n_markers, compactness=compact/1000.0)
    img_watershed = mark_boundaries(img, segments_watershed)
    homogenized_watershed = label2rgb(segments_watershed, img, kind='avg')
    
    affichage_auto_simple([img, img_watershed, homogenized_watershed], ["Original", f"Watershed clusters: {np.unique(segments_watershed).size}", "Homogenized"])

interactive(children=(Dropdown(description='Image:', index=5, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

## <span style="color: DodgerBlue;text-decoration: underline">II.3 Superpixels</span>

*Many existing algorithms in computer vision use the pixel-grid as the underlying representation. The pixel-grid, however, is not a natural representation of visual scenes. It is rather an "artifact" of a digital imaging process. It would be more natural, and presumably more efficient, to work with perceptually meaningful entities obtained from a low-level grouping process.*

*Superpixels are one answer to this problem: a superpixel can be defined as a group of pixels that share common characteristics (like pixel intensity ) :*
* _They carry more information than pixels._
* _They have a perceptual meaning since pixels belonging to a given superpixel share similar visual properties._
* _They provide a convenient and compact representation of images that can be very useful for computationally demanding problems._

#### Avec `Skimage`: SLIC (Simple Linear Iterative Clustering)

Une implémentation classique du principe des superpixels est l'algorithme **SLIC** : *this algorithm uses a machine learning algorithm called K-Means under the hood. It takes in all the pixel values of the image and tries to separate them out into the given number of sub-regions. It generates superpixels by clustering pixels based on their color similarity and proximity in the image plane.*

*SLIC has 3 main paramters:*
* _`n_segments` controls the (approximately desired number of regions in the output image._
* _`compactness` controls the balances between color proximity and space proximity. Higher values give more weight to space proximity, making superpixel shapes more square/cubic._
* _`sigma` controls the width of the Gaussian smoothing as preprocessing (zero means no smoothing)._


[Documentation SLIC](https://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic)

In [6]:
images_dropdown = widgets.Dropdown(options=[f for f in os.listdir(img_path) if os.path.isfile(os.path.join(img_path, f))],
    value='coins2.jpg', description='Image:')
segments_slider = widgets.IntSlider(min=10, max=300, step=10, value=100, continuous_update=False)
compactness_slider = widgets.IntSlider(min=1, max=30, step=1, value=10, continuous_update=False)
sigma_slider = widgets.IntSlider(min=1, max=15, step=1, value=5, continuous_update=False)

@interact
def superpixels_slic(image=images_dropdown, segments=segments_slider, compact=compactness_slider, sigma_val=sigma_slider):
    img = np.array(Image.open(img_path + image)).astype("uint8")
    
    segments_slic = slic(img, n_segments=segments, compactness=compact, sigma=sigma_val)
    img_slic = mark_boundaries(img, segments_slic)
    homogenized_slic = label2rgb(img_slic, img, kind='avg')
    
    affichage_auto_simple([img, img_slic, homogenized_slic], ["Original", f"SLIC clusters: {np.unique(segments_slic).size}", "Homogenized"])

interactive(children=(Dropdown(description='Image:', index=5, options=('adapt1.jpg', 'adapt2.jfif', 'barbara.j…

### <span style="color:crimson">**[<u>Exercice</u>]** A vous de jouer:</span>
***
<div style="color:DarkSlateBlue">
    
1. **Implémentez une méthode interactive permettant de sélectionner l'un des 3 algorithmes de clustering pour l'appliquer à l'image `cells.jpg`.**
    
    
2. **Optimisez les paramètres de la méthode afin d'obtenir la meilleure segmentation possible, et servez vous des clusters génrés pour homogénéiser l'image.**
    
    
3. **A partir de l'image homogénéisée, reprenez le code de l'exercice I. de ce TP afin d'obtenir un meilleur masque de segmentation.**
    
    
</div>

In [29]:
# > Emplacement exercice <



<div style="color:Navy"> 

***
# Fin du TP5
***
    
</div>